# Week 5 Assignment  Data Science Automation  Due 7-31-2022
# Mary J Hollon 

Using our prepared churn data from week 2:

- Use pycaret to find an ML algorithm that performs best on the data
- Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
- your Python file/function should print out the predictions for new data (new_churn_data.csv)
- the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

- Optional challenges:

- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's input() function, the click package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1.

## Load data

In [1]:
# Loading the data
import pandas as pd

data = pd.read_csv("prepared_churn_data.csv")
# checking the data frame
data


,customerID,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,charge_per_tenure
0,7590-VHVEG,1,0,0,3,29.85,29.85,0,29.850000
1,5575-GNVDE,34,1,1,2,56.95,1889.50,0,55.573529
2,3668-QPYBK,2,1,0,2,53.85,108.15,1,54.075000
3,7795-CFOCW,45,0,1,1,42.30,1840.75,0,40.905556
4,9237-HQITU,2,1,0,3,70.70,151.65,1,75.825000
...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,24,1,1,2,84.80,1990.50,0,82.937500
7039,2234-XADUH,72,1,1,0,103.20,7362.90,0,102.262500
7040,4801-JZAZL,11,0,0,3,29.60,346.45,0,31.495455
7041,8361-LTMKD,4,1,0,2,74.40,306.60,1,76.650000


# AutoML with pycaret

My next step is to use pycaret for autoML. I had to install pycaret-nightly from a command prompt :
`conda install -c conda-forge pycaret-nightly -y`. This seemed to have fixed my issue

In [2]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

Next, we can setup our autoML:Remember to hit <Enter> to accept the data and proceed

In [3]:
automl = setup(data, target='Churn')

,Description,Value
0,session_id,6398
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(7043, 9)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,4
8,Ordinal Features,False
9,High Cardinality Features,False


This will ask us to check if the datatypes of the input data are correct. In this case, they seem fine. 

In [5]:
automl[10]

Next up, we simply run the autoML to find the best model and sort it by AUC. Ideally AUC will approach 1

In [6]:
best_model = compare_models(sort="AUC")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7905,0.8375,0.4510,0.6641,0.5361,0.4072,0.4205,9.8210
gbc,Gradient Boosting Classifier,0.7895,0.8344,0.4532,0.6615,0.5360,0.4061,0.4194,12.5000
ada,Ada Boost Classifier,0.7909,0.8337,0.5136,0.6405,0.5683,0.4328,0.4385,3.5280
lr,Logistic Regression,0.7899,0.8327,0.5205,0.6334,0.5708,0.4335,0.4376,22.5070
rf,Random Forest Classifier,0.7888,0.8248,0.4260,0.6713,0.5196,0.3930,0.4106,4.1480
et,Extra Trees Classifier,0.7874,0.8216,0.4177,0.6705,0.5136,0.3868,0.4052,6.6230
lightgbm,Light Gradient Boosting Machine,0.7751,0.8192,0.4789,0.6073,0.5332,0.3880,0.3942,0.9760
lda,Linear Discriminant Analysis,0.7314,0.8153,0.0000,0.0000,0.0000,0.0000,0.0000,52.9400
nb,Naive Bayes,0.6430,0.8076,0.8957,0.4230,0.5744,0.3297,0.3995,0.3740
xgboost,Extreme Gradient Boosting,0.7649,0.8075,0.4796,0.5773,0.5214,0.3680,0.3722,24.1390


The "boost" classifiers are all at the top with CatBoost Classifier being the best performing model based on AUC. The Naive Bayes Model has the Best Recall and good AUC. 

In [7]:
best_model

The best model based on AUC is Cat Boost Cassifier
We can now use the model to make predictions. 

In [8]:
data.iloc[-2:-1].shape

(1, 9)

We are selecting the last row, but using the indexing `[-2:-1]` to make it a 2D array instead of 1D (which throws an error). 
Use Cat Boost to predict the probability class on the last row of the data frame 

In [9]:
predict_model(best_model, data.iloc[-2:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,1.0000,0,1.0000,1.0000,1.0000,nan,0.0000


,customerID,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,charge_per_tenure,Label,Score
7041,8361-LTMKD,4,1,0,2,74.4,306.6,1,76.65,1,0.5627


We can see this creates a new column, 'Score', with the probability of class 1. It also creates a 'Label' column with the predicted label, where it rounds up if score is >= 0.5 (greater than or equal to 0.5).

Cat Boost predicts that this customer will churn ( 1=Yes, 0=No ) with a probability score of .56

## Saving and loading our model

Next, we want to save our trained model so we can use it in a Python file later. 

In [19]:
save_model(best_model, 'cbc')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Churn',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strate...
                 ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                 ('cluster_all', 'passthrough'),
                 ('dummy', Dummify(target='Churn')),
                 ('fix_perfect', Remove_100(target='Churn')),
                 ('clean_names', Cl

In [22]:
loaded_cbc = load_model('cbc')

Transformation Pipeline and Model Successfully Loaded


In [23]:
new_data = data.iloc[-2:-1].copy()
new_df=new_data.drop('Churn', axis=1) 
loaded_cbc.predict(new_df)

array([1], dtype=int64)

In [24]:
predict_model(loaded_cbc, new_df)

,customerID,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,charge_per_tenure,Label,Score
7041,8361-LTMKD,4,1,0,2,74.4,306.6,76.65,1,0.5627


# Making a Python module to make predictions

We can now use this model in a Python file to take in new data and make a prediction. We will first need to compose a Python file. I chose to keep it simple in Notepad.

The inported Python File is below:

In [26]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd

from pycaret.classification import predict_model, load_model


df = pd.read_csv("new_churn_data.csv")
    

model = load_model('cbc')

predictions = predict_model(model, data=df)

    
print('predictions:')

print(predictions)

We can test out running the file with the Jupyter "magic" command %run:

In [27]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded
predictions:
   customerID  tenure  PhoneService  Contract  PaymentMethod  MonthlyCharges  \
0  9305-CKSKC      22             1         0              2           97.40   
1  1452-KNGVK       8             0         1              1           77.30   
2  6723-OKKJM      28             1         0              0           28.25   
3  7832-POPKP      62             1         0              2          101.70   
4  6348-TACGU      10             0         0              1           51.15   

   TotalCharges  charge_per_tenure  Label   Score  
0        811.70          36.895455      0  0.8694  
1       1701.95         212.743750      0  0.6151  
2        250.90           8.960714      0  0.9128  
3       3106.56          50.105806      0  0.9339  
4       3440.97         344.097000      1  0.5794  


Cat Boost CLassifier predicts that 4 customers will NOT churn and only 1 will churn. The accuracy is just under 80 % and the recall score is low so it may not be predicting very well. The AUC was 83.7% with ideal being 1. 

# Assignment Summary

Before I was able to start this assignment, I had to fix my issue with Jupyter Notebook not opening. It simply stopped opening from Anaconda Navigtor and would not open at all. So I did what I didn't want to do and that was to uninstall and reinstall Anaconda Navigator. It got Jupyter Notebook working, but I soon learned that some packages were not working as intended. I had difficulty getting pycaret to install. In order to make it work, I had to downgrade my version of Python, and install the correct versions of Jinja2, and MarkeUpSafe. FInally, I read that I should try to install pycaret-nightly and it worked ! Since I am now working with a new version of Anaconda Navigator, all of my packages are upgraded or the most recent. I hope I don't experience any future issues.

For this assignment, I used the prepped churn data from week 2. The data was loaded and inspected to ensure it loaded correctly. Next it was loaded into automl and fitted to 16 different models. I used Area under the curve (AUC) and looked at recall. The best model was CatBoost. It was saved to the best model and used to predict the churn on the last row of the data set. 

Next a simple python script was coded using Notepad and saved as a python file. The new_churn_data was read and executed in the main Jupyter Notebook file and the resulting predictions were printed out. The CatBoost model is not very robust, and had a low accuracy. It did predict 1 customer to churn in the new data set. The Naive Bayes had a better recall but a lower precision. Overall, none of the models were scoring highly in any of the metrics. 

A GitHub Repository was created and the Jupyter Notebook and Python were uploaded on GitHub. The resulting link was submitted in the submission file. 

